In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# 예시 CPU 데이터

amd_data = pd.read_csv('amd.csv', encoding='euc-kr')
intel_data = pd.read_csv('intel.csv', encoding='euc-kr')

# Price를 숫자로 변환
intel_data['Price'] = intel_data['Price'].str.replace(',', '').astype(float)
amd_data['Price'] = amd_data['Price'].str.replace(',', '').astype(float)
# 시네벤치R23(싱글)과 시네벤치R23(멀티)를 숫자로 변환
intel_data['시네벤치R23(싱글)'] = intel_data['시네벤치R23(싱글)'].astype(float)
intel_data['시네벤치R23(멀티)'] = intel_data['시네벤치R23(멀티)'].astype(float)
intel_data['SinglePerformance'] = intel_data['시네벤치R23(싱글)'] * 1000 / intel_data['Price']
intel_data['MultiPerformance'] = intel_data['시네벤치R23(멀티)'] * 1000 / intel_data['Price']
amd_data['시네벤치R23(싱글)'] = amd_data['시네벤치R23(싱글)'].astype(float)
amd_data['시네벤치R23(멀티)'] = amd_data['시네벤치R23(멀티)'].astype(float)
amd_data['SinglePerformance'] = amd_data['시네벤치R23(싱글)'] * 1000 / amd_data['Price']
amd_data['MultiPerformance'] = amd_data['시네벤치R23(멀티)'] * 1000 / amd_data['Price']

print(intel_data[['Product Name', 'SinglePerformance', 'MultiPerformance']])
print(amd_data[['Product Name', 'SinglePerformance', 'MultiPerformance']])

intel_single_avg = intel_data['SinglePerformance'].mean()
intel_multi_avg = intel_data['MultiPerformance'].mean()
amd_single_avg = amd_data['SinglePerformance'].mean()
amd_multi_avg = amd_data['MultiPerformance'].mean()

print("Intel CPU SinglePerformance 평균값:", intel_single_avg)
print("Intel CPU MultiPerformance 평균값:", intel_multi_avg)
print("AMD CPU SinglePerformance 평균값:", amd_single_avg)
print("AMD CPU MultiPerformance 평균값:", amd_multi_avg)

                         Product Name  SinglePerformance  MultiPerformance
0    인텔 코어i5-14세대 14400F (랩터레이크 리프레시)           6.303039         62.692348
1    인텔 코어i7-14세대 14700K (랩터레이크 리프레시)           3.516375         59.175037
2         인텔 코어i5-12세대 12400F (엘더레이크)          10.892433         82.802850
3    인텔 코어i9-14세대 14900K (랩터레이크 리프레시)           2.798397         49.338806
4     인텔 코어i5-14세대 14500 (랩터레이크 리프레시)           5.578310         63.585232
5         인텔 코어i5-13세대 13600K (랩터레이크)           5.072390         59.804574
6   인텔 코어i7-14세대 14700KF (랩터레이크 리프레시)           3.890027         63.868990
7         인텔 코어i5-13세대 13400F (랩터레이크)           7.520282         61.461180
8    인텔 코어i5-14세대 14600K (랩터레이크 리프레시)           4.641095         56.435621
9    인텔 코어i7-14세대 14700F (랩터레이크 리프레시)           4.002220         61.420970
10    인텔 코어i7-14세대 14700 (랩터레이크 리프레시)           3.648340         56.082909
11    인텔 코어i3-14세대 14100 (랩터레이크 리프레시)           8.454071         45.294892
12        인텔 코어i3-12세대 12

In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 전처리
def preprocess_data(data):
    # 문자열 데이터를 원-핫 인코딩으로 변환
    data_encoded = pd.get_dummies(data, columns=['Socket', '메모리 타입'])

    # 'SinglePerformance'와 'MultiPerformance'를 추가
    data_encoded['SinglePerformance'] = (data_encoded['시네벤치R23(싱글)'] * 1000) / data_encoded['Price']
    data_encoded['MultiPerformance'] = (data_encoded['시네벤치R23(멀티)'] * 1000) / data_encoded['Price']


    # '시네벤치R23(싱글)', '시네벤치R23(멀티)' 열 제거
    data_encoded = data_encoded.drop(columns=['시네벤치R23(싱글)', '시네벤치R23(멀티)'])

    # 'Product Name' 열 저장
    data_encoded['Product Name'] = data['Product Name'].values

    return data_encoded

# 선택한 CPU의 모든 정보를 반환하는 함수
def get_cpu_details(cpu_data, product_name):
    return cpu_data[cpu_data['Product Name'] == product_name]

amd_data_processed = preprocess_data(amd_data)
intel_data_processed = preprocess_data(intel_data)

# 현재 열 목록 출력
def print_column_names(df):
    print("현재 열 목록:")
    print(df.columns.tolist())

# 선택한 CPU의 모든 정보 중 일부 정보만 반환하는 함수
def get_filtered_cpu_details(cpu_data, product_name, brand):
    # 현재 열 목록 출력
    print_column_names(cpu_data)
    # 브랜드에 따라 필요한 정보 열 정의
    if brand == 'Intel':
        required_columns = [
            'Product Name',
            'Socket_인텔(소켓1200)',
            'Socket_인텔(소켓1700)',
            '메모리 타입_DDR4',
            '메모리 타입_DDR5, DDR4'
        ]
    elif brand == 'AMD':
        required_columns = [
            'Product Name',
            'Socket_AMD(소켓AM4)',
            'Socket_AMD(소켓AM5)',
            '메모리 타입_DDR4',
            '메모리 타입_DDR5'
        ]
    else:
        raise ValueError("지원하지 않는 브랜드입니다.")

    # 선택한 CPU의 정보 추출
    cpu_info = cpu_data[cpu_data['Product Name'] == product_name]

    # 필요한 정보만 추출
    return cpu_info[required_columns]

# 유사도 계산 함수
def compute_similarity(cpu_data, user_profile):
    # 사용자 프로필을 수치형으로 변환
    user_profile_encoded = {
        'Price': user_profile['Price'],
        'SinglePerformance': user_profile['SinglePerformance'],
        'MultiPerformance': user_profile['MultiPerformance'],
    }
    user_profile_encoded['Value'] = user_profile_encoded['Price'] / (user_profile_encoded['SinglePerformance'] + user_profile_encoded['MultiPerformance'])

    # 모든 원-핫 인코딩 열이 포함된 벡터 생성
    all_columns = cpu_data.columns
    user_vector = np.array([user_profile_encoded.get(col, 0) for col in all_columns if col != 'Product Name']).reshape(1, -1)
    cpu_vectors = cpu_data[[col for col in all_columns if col != 'Product Name']].values
    similarities = cosine_similarity(user_vector, cpu_vectors)
    cpu_data['Similarity'] = similarities[0]
    return cpu_data

# 유저 프로필 설정
user_purpose = input('용도를 입력하세요(게임용, 사무용, 영상편집):')
user_budget = int(input('예산을 입력하세요:'))

# 용도별 사전 정의된 프로필
profiles = {
    '게임용': {
        'Price': user_budget,
        'SinglePerformance': 7,  # 예시 값
        'MultiPerformance': 70   # 예시 값
    },
    '사무용': {
        'Price': user_budget,
        'SinglePerformance': 6,   # 예시 값
        'MultiPerformance': 60   # 예시 값
    },
    '영상편집': {
        'Price': user_budget,
        'SinglePerformance': 6,  # 예시 값
        'MultiPerformance': 100   # 예시 값
    }
}

# 입력된 용도에 따라 프로필 선택
user_profile = profiles.get(user_purpose, profiles['사무용'])  # 기본값은 '사무용'

# 유사도 계산 및 추천 생성
cpu_data_AMD = compute_similarity(amd_data_processed, user_profile)
cpu_data_INTEL = compute_similarity(intel_data_processed, user_profile)

용도를 입력하세요(게임용, 사무용, 영상편집):게임용
예산을 입력하세요:500000


In [48]:
selected_cpu_info = []
if user_purpose == '영상편집':
    recommendations_INTEL = cpu_data_INTEL[cpu_data_INTEL['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)
    print("추천 Intel CPU 리스트:")
    recommendations_INTEL.reset_index(drop=True, inplace=True)
    for idx, row in recommendations_INTEL.head().iterrows():
        print(f"{idx + 1}. {row['Product Name']} - Price: {row['Price']}, SinglePerformance: {row['SinglePerformance']:.2f}, MultiPerformance: {row['MultiPerformance']:.2f}")

    selected_index = int(input("선택할 Intel CPU의 번호를 입력하세요:")) - 1
    if 0 <= selected_index < len(recommendations_INTEL):
        selected_cpu_name = recommendations_INTEL.iloc[selected_index]['Product Name']
        selected_cpu_info = get_filtered_cpu_details(intel_data_processed, selected_cpu_name, 'Intel')
        print("\n선택한 Intel CPU의 정보:")
        print(selected_cpu_info)
    else:
        print("유효하지 않은 번호입니다.")
else:
    # 사용자에게 선택할 브랜드를 묻기
    brand_choice = input("AMD 또는 Intel 중 어느 브랜드의 CPU를 선택하시겠습니까? (AMD/Intel):").strip().lower()

    if brand_choice == 'amd':
        recommendations_AMD = cpu_data_AMD[cpu_data_AMD['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)
        print("추천 AMD CPU 리스트:")
        recommendations_AMD.reset_index(drop=True, inplace=True)
        for idx, row in recommendations_AMD.head().iterrows():
            print(f"{idx + 1}. {row['Product Name']} - Price: {row['Price']}, SinglePerformance: {row['SinglePerformance']:.2f}, MultiPerformance: {row['MultiPerformance']:.2f}")

        selected_index = int(input("선택할 AMD CPU의 번호를 입력하세요:")) - 1
        if 0 <= selected_index < len(recommendations_AMD):
            selected_cpu_name = recommendations_AMD.iloc[selected_index]['Product Name']
            selected_cpu_info = get_filtered_cpu_details(amd_data_processed, selected_cpu_name, 'AMD')
            print("\n선택한 AMD CPU의 정보:")
            print(selected_cpu_info)
        else:
            print("유효하지 않은 번호입니다.")

    elif brand_choice == 'intel':
        recommendations_INTEL = cpu_data_INTEL[cpu_data_INTEL['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)
        print("추천 Intel CPU 리스트:")
        recommendations_INTEL.reset_index(drop=True, inplace=True)
        for idx, row in recommendations_INTEL.head().iterrows():
            print(f"{idx + 1}. {row['Product Name']} - Price: {row['Price']}, SinglePerformance: {row['SinglePerformance']:.2f}, MultiPerformance: {row['MultiPerformance']:.2f}")

        selected_index = int(input("선택할 Intel CPU의 번호를 입력하세요:")) - 1
        if 0 <= selected_index < len(recommendations_INTEL):
            selected_cpu_name = recommendations_INTEL.iloc[selected_index]['Product Name']
            selected_cpu_info = get_filtered_cpu_details(intel_data_processed, selected_cpu_name, 'Intel')
            print("\n선택한 Intel CPU의 정보:")
            print(selected_cpu_info)
        else:
            print("유효하지 않은 번호입니다.")

    else:
        print("유효하지 않은 브랜드입니다.")

AMD 또는 Intel 중 어느 브랜드의 CPU를 선택하시겠습니까? (AMD/Intel):AMD
추천 AMD CPU 리스트:
1. AMD 라이젠7-5세대 7700 (라파엘) - Price: 364160.0, SinglePerformance: 5.27, MultiPerformance: 51.34
2. AMD 라이젠7-4세대 5700X3D (버미어) - Price: 285390.0, SinglePerformance: 5.35, MultiPerformance: 46.58
3. AMD 라이젠7-5세대 8700G (피닉스) - Price: 392000.0, SinglePerformance: 4.35, MultiPerformance: 45.04
4. AMD 라이젠5-4세대 5600G (세잔) - Price: 251690.0, SinglePerformance: 5.75, MultiPerformance: 41.67
5. AMD 라이젠5-5세대 7600X (라파엘) - Price: 302380.0, SinglePerformance: 6.56, MultiPerformance: 50.29
선택할 AMD CPU의 번호를 입력하세요:1
현재 열 목록:
['Product Name', 'Price', 'Integrated Graphics', 'SinglePerformance', 'MultiPerformance', 'Socket_AMD(소켓AM4)', 'Socket_AMD(소켓AM5)', 'Socket_AMD(소켓sWRX8)', '메모리 타입_DDR4', '메모리 타입_DDR5', 'Similarity']

선택한 AMD CPU의 정보:
               Product Name  Socket_AMD(소켓AM4)  Socket_AMD(소켓AM5)  \
12  AMD 라이젠7-5세대 7700 (라파엘)              False               True   

    메모리 타입_DDR4  메모리 타입_DDR5  
12        False         True 

In [49]:
print(selected_cpu_info)

               Product Name  Socket_AMD(소켓AM4)  Socket_AMD(소켓AM5)  \
12  AMD 라이젠7-5세대 7700 (라파엘)              False               True   

    메모리 타입_DDR4  메모리 타입_DDR5  
12        False         True  


In [55]:
import pandas as pd
gpu_data = pd.read_csv('gpu.csv', encoding='utf-8')

# Price를 숫자로 변환
gpu_data['Price'] = gpu_data['Price'].str.replace(',', '').astype(float)
print(gpu_data[['Product Name', 'Price']])
print(gpu_data['Price'].mean())

                                         Product Name      Price
0             이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB   418030.0
1       ZOTAC GAMING 지포스 RTX 4060 TWIN Edge OC D6 8GB   414880.0
2          이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB   557590.0
3            MSI 지포스 RTX 4060 Ti 벤투스 2X 블랙 OC D6 16GB   618870.0
4    MSI 지포스 RTX 4070 SUPER 게이밍 X 슬림 D6X 12GB 트라이프로져3   977910.0
5                MSI 지포스 RTX 4060 벤투스 2X 블랙 OC D6 8GB   426990.0
6   MSI 지포스 RTX 4070 Ti SUPER 게이밍 X 슬림 D6X 16GB 트라...  1244100.0
7               XFX 라데온 RX 6800 SWFT 319 CORE D6 16GB   500200.0
8              갤럭시 GALAX 지포스 RTX 4060 WHITE OC D6 8GB   410400.0
9               이엠텍 지포스 RTX 4060 MIRACLE WHITE D6 8GB   426960.0
10   MSI 지포스 RTX 4080 SUPER 게이밍 X 슬림 D6X 16GB 트라이프로져3  1658860.0
11           MSI 지포스 RTX 4090 슈프림 X D6X 24GB 트라이프로져3S  2998810.0
12             ASUS DUAL 라데온 RX 7600 O8G V2 OC D6 8GB   357980.0
13  GIGABYTE 지포스 RTX 4070 SUPER WINDFORCE OC D6X 1...   904890.0
14                 MSI 지포

In [74]:
# 사용자 입력 받기
user_budget = int(input('예산을 입력하세요:'))
user_purpose = input('용도를 입력하세요(게임용, 사무용, 영상편집):')

# 용도별 사전 정의된 프로필
profiles = {
    '게임용': {
        'Price': 1000000
    },
    '사무용': {
        'Price': 800000
    },
    '영상편집': {
        'Price': 800000
    }
}

# 입력된 용도에 따라 프로필 선택
user_profile = profiles.get(user_purpose, profiles['사무용'])  # 기본값은 '사무용'

# 유사도 계산 함수
def compute_similarity(gpu_data, user_profile):
    features = ['Price']
    user_price = user_profile['Price']

    # 유사도를 절대 가격 차이로 계산
    gpu_data['Similarity'] = 1 / (1 + abs(gpu_data['Price'] - user_price))
    return gpu_data

# 유사도 계산
gpu_data = compute_similarity(gpu_data, user_profile)

# 추천 생성 (상위 5개)
recommendations = gpu_data[gpu_data['Price'] <= user_budget].sort_values(by='Similarity', ascending=False).head(5)

# 번호를 임의로 부여하여 추가
recommendations.reset_index(drop=True, inplace=True)
recommendations.index = recommendations.index + 1  # 인덱스를 1부터 시작하도록 조정

print("추천 GPU 리스트:")
for idx, row in recommendations[['Product Name', 'Price']].iterrows():
    # idx + 1로 번호를 1부터 시작하도록 조정
    print(f"{idx}. {row['Product Name']} - Price: {row['Price']}")

# 사용자에게 선택하라고 요청
selection = int(input('선택할 GPU의 번호를 입력하세요 (1-5):'))

# 선택한 GPU의 정보를 출력
if 1 <= selection <= len(recommendations):
    selected_gpu = recommendations.loc[selection]
    print("\n선택한 GPU의 정보:")
    print(selected_gpu)
else:
    print("유효하지 않은 번호입니다.")

예산을 입력하세요:700000
용도를 입력하세요(게임용, 사무용, 영상편집):게임용
추천 GPU 리스트:
1. ASUS TUF Gaming 라데온 RX 7800 XT O16G OC D6 16GB - Price: 680840.0
2. MSI 지포스 RTX 4060 Ti 벤투스 2X 블랙 OC D6 16GB - Price: 618870.0
3. 이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB - Price: 557590.0
4. XFX 라데온 RX 6800 SWFT 319 CORE D6 16GB - Price: 500200.0
5. ASUS DUAL 지포스 RTX 4060 O8G OC D6 8GB White - Price: 447000.0
선택할 GPU의 번호를 입력하세요 (1-5):1

선택한 GPU의 정보:
Product Name    ASUS TUF Gaming 라데온 RX 7800 XT O16G OC D6 16GB
Price                                                 680840.0
Power_Type                                                750W
가로(길이)                                                 319.8mm
Similarity                                            0.000003
Name: 1, dtype: object


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시: 사용자로부터 용량, DDR 유형, 예산 입력받기
user_capacity = int(input("RAM 용량을 입력하세요 (GB 단위): "))
user_ddr = int(input("DDR 유형을 입력하세요 (예: 3, 4, 5): "))
user_budget = int(input("예산을 입력하세요 (달러 단위): "))

# 사용자 프로필 생성
user_profile = {
    'Capacity': user_capacity,
    'DDR': user_ddr,
    'Price': user_budget
}

# 예시 RAM 데이터
data = {
    'RAM Model': ['Corsair Vengeance LPX 16GB', 'G.SKILL Ripjaws V 32GB', 'Kingston HyperX Fury 8GB', 'Crucial Ballistix 16GB'],
    'Capacity': [16, 32, 8, 16],
    'DDR': [4, 4, 3, 3],
    'Price': [80, 160, 40, 70],
    'Manufacturer': ['Corsair', 'G.SKILL', 'Kingston', 'Crucial']
}

ram_data = pd.DataFrame(data)
ram_data.to_csv('ram_data.csv', index=False)

# 3. 유사도 계산
def compute_similarity(ram_data, user_profile):
    features = ['Capacity', 'DDR', 'Price']
    user_vector = np.array([user_profile[feature] for feature in features]).reshape(1, -1)
    ram_vectors = ram_data[features].values
    similarities = cosine_similarity(user_vector, ram_vectors)
    ram_data['Similarity'] = similarities[0]
    return ram_data

ram_data = compute_similarity(ram_data, user_profile)

# 4. 추천 생성
recommendations = ram_data[ram_data['Price'] <= user_profile['Price']].sort_values(by='Similarity', ascending=False)

print("추천 RAM 리스트:")
print(recommendations[['RAM Model', 'Similarity']].head())


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시: 사용자로부터 CPU 세대, M.2 지원 여부, DDR 유형 입력받기
user_cpu_generation = int(input("CPU 세대를 입력하세요: "))
user_m2_support = int(input("M.2 지원 여부를 입력하세요 (지원: 1, 미지원: 0): "))
user_ddr = int(input("DDR 유형을 입력하세요 (예: 3, 4, 5): "))

# 사용자 프로필 생성
user_profile = {
    'CPU Generation': user_cpu_generation,
    'M.2 Support': user_m2_support,
    'DDR': user_ddr
}

# 예시 메인보드 데이터
data = {
    'Mainboard Model': ['ASUS ROG Strix Z490-E', 'MSI MPG Z390 Gaming Pro', 'Gigabyte B450 AORUS M', 'ASRock B460M Pro4'],
    'CPU Generation': [10, 9, 3, 10],
    'M.2 Support': [1, 1, 1, 0],
    'DDR': [4, 4, 4, 4],
    'Price': [250, 180, 100, 90],
    'Manufacturer': ['ASUS', 'MSI', 'Gigabyte', 'ASRock']
}

mainboard_data = pd.DataFrame(data)
mainboard_data.to_csv('mainboard_data.csv', index=False)

# 3. 유사도 계산
def compute_similarity(mainboard_data, user_profile):
    features = ['CPU Generation', 'M.2 Support', 'DDR']
    user_vector = np.array([user_profile[feature] for feature in features]).reshape(1, -1)
    mainboard_vectors = mainboard_data[features].values
    similarities = cosine_similarity(user_vector, mainboard_vectors)
    mainboard_data['Similarity'] = similarities[0]
    return mainboard_data

mainboard_data = compute_similarity(mainboard_data, user_profile)

# 4. 추천 생성
recommendations = mainboard_data.sort_values(by='Similarity', ascending=False)

print("추천 메인보드 리스트:")
print(recommendations[['Mainboard Model', 'Similarity']].head())

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 데이터
data = {
    'Case Model': ['NZXT H510', 'Cooler Master Q300L', 'Fractal Design Meshify C', 'Corsair 275R Airflow'],
    'Price': [100, 50, 90, 80],
    'CPU Form Factor': ['ATX', 'Micro-ATX', 'ATX', 'ATX']
}

case_data = pd.DataFrame(data)

# 예시: 사용자로부터 가격 및 CPU 폼팩터 입력받기
user_price = float(input("가격을 입력하세요: "))
user_cpu_form_factor = input("CPU 폼팩터를 입력하세요 (예: ATX, Micro-ATX): ")

# 사용자 프로필 생성
user_profile = {
    'Price': user_price,
    'CPU Form Factor': user_cpu_form_factor
}

# 1. 가격 표준화
scaler = StandardScaler()
case_data['Normalized Price'] = scaler.fit_transform(case_data[['Price']])

# 2. CPU 폼팩터 원핫 인코딩
case_data = pd.get_dummies(case_data, columns=['CPU Form Factor'])

# 3. 사용자 프로필 데이터 변환
user_profile['Normalized Price'] = scaler.transform([[user_profile['Price']]])[0][0]
user_profile_vector = [user_profile['Normalized Price']] + [0] * (len(case_data.columns) - 3)

if 'CPU Form Factor_' + user_profile['CPU Form Factor'] in case_data.columns:
    index = case_data.columns.get_loc('CPU Form Factor_' + user_profile['CPU Form Factor'])
    user_profile_vector[index - 1] = 1  # Adjusted index for user vector

user_vector = np.array(user_profile_vector).reshape(1, -1)
case_vectors = case_data.drop(columns=['Case Model', 'Price']).values

# 4. 유사도 계산
similarities = cosine_similarity(user_vector, case_vectors)
case_data['Similarity'] = similarities[0]

# 5. 추천 생성
recommendations = case_data.sort_values(by='Similarity', ascending=False)

print("추천 케이스 리스트:")
print(recommendations[['Case Model', 'Similarity']])

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 데이터
data = {
    'Power Supply Model': ['Corsair RM750x', 'EVGA SuperNOVA 750 G5', 'Seasonic FOCUS GX-650', 'Cooler Master MWE Gold 650 V2'],
    'Price': [120, 110, 90, 80],
    'ATX Support': ['Yes', 'Yes', 'Yes', 'No'],
    'CPU Type': ['External', 'External', 'External', 'Internal']
}

psu_data = pd.DataFrame(data)

# 사용자 입력
user_price = 100
user_atx_support = 'Yes'
user_cpu_type = 'External'

# 사용자 프로필 생성
user_profile = {
    'Price': user_price,
    'ATX Support': user_atx_support,
    'CPU Type': user_cpu_type
}

# 1. 가격 표준화
scaler = StandardScaler()
psu_data['Normalized Price'] = scaler.fit_transform(psu_data[['Price']])

# 2. ATX 지원 여부 및 CPU 타입 원핫 인코딩
psu_data = pd.get_dummies(psu_data, columns=['ATX Support', 'CPU Type'])

# 3. 사용자 프로필 데이터 변환
user_profile['Normalized Price'] = scaler.transform([[user_profile['Price']]])[0][0]

# 유저 프로필 벡터 크기 설정 (열 이름에 따라)
user_profile_vector = [user_profile['Normalized Price']]

# 원핫 인코딩된 열에 맞추어 사용자 프로필 벡터 업데이트
for column in psu_data.columns[2:]:  # 'Power Supply Model'과 'Price'를 제외한 모든 열
    if column == 'Normalized Price':
        continue
    if column == 'ATX Support_' + user_profile['ATX Support'] or column == 'CPU Type_' + user_profile['CPU Type']:
        user_profile_vector.append(1)
    else:
        user_profile_vector.append(0)

user_vector = np.array(user_profile_vector).reshape(1, -1)
psu_vectors = psu_data.drop(columns=['Power Supply Model', 'Price']).values

# 4. 유사도 계산
similarities = cosine_similarity(user_vector, psu_vectors)
psu_data['Similarity'] = similarities[0]

# 5. 추천 생성
recommendations = psu_data.sort_values(by='Similarity', ascending=False)

print("추천 파워 서플라이 리스트:")
print(recommendations[['Power Supply Model', 'Similarity']])


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 예시 SSD 데이터
ssd_data = {
    'SSD Model': ['Samsung 970 EVO', 'Western Digital WD Black', 'Crucial MX500', 'Samsung 860 EVO'],
    'Capacity (GB)': [500, 1000, 500, 1000],
    'M.2': ['Yes', 'Yes', 'No', 'No'],
    'Price': [150, 200, 100, 120]
}

ssd_df = pd.DataFrame(ssd_data)

# 사용자 입력
user_capacity = 500  # GB
user_m2 = 'Yes'

# 사용자 프로필 생성
user_profile = {
    'Capacity (GB)': user_capacity,
    'M.2': user_m2
}

# 1. 가격 표준화
scaler = StandardScaler()
ssd_df['Normalized Price'] = scaler.fit_transform(ssd_df[['Price']])

# 2. M.2 지원 여부 원핫 인코딩
ssd_df = pd.get_dummies(ssd_df, columns=['M.2'])

# 3. 사용자 프로필 데이터 변환
user_profile['Normalized Price'] = scaler.transform([[user_profile['Capacity (GB)']]])[0][0]

# 4. 사용자 프로필 벡터 생성
user_profile_vector = [user_profile['Capacity (GB)'], user_profile['Normalized Price']]

# 원핫 인코딩된 열에 맞추어 사용자 프로필 벡터 업데이트
m2_columns = [col for col in ssd_df.columns if col.startswith('M.2')]
for column in m2_columns:
    if column == 'M.2_' + user_profile['M.2']:
        user_profile_vector.append(1)
    else:
        user_profile_vector.append(0)

user_vector = np.array(user_profile_vector).reshape(1, -1)

# 5. SSD 벡터 생성
ssd_vectors = ssd_df.drop(columns=['SSD Model', 'Price']).values

# 6. 유사도 계산
similarities = cosine_similarity(user_vector, ssd_vectors)
ssd_df['Similarity'] = similarities[0]

# 7. 추천 생성
recommendations = ssd_df.sort_values(by='Similarity', ascending=False)

print("추천 SSD 리스트:")
print(recommendations[['SSD Model', 'Similarity']])
